# Data processer
## Proyecto final -- Optimización y Simulación
### Alejandro de Haro, Jose María Benítez, Daniel Escobosa, Pablo Berástegui

In [55]:
import pandas as pd

In [56]:
df_players = pd.read_csv('../data/male_players.csv')

C:\Users\teach\AppData\Local\Temp\ipykernel_29480\910873479.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_players = pd.read_csv('../data/male_players.csv')


In [57]:
df_players = df_players[df_players['fifa_version'] == 18.0]

In [58]:
# tipos de cada columna
for col in df_players.columns:
    print(f"{col}: {df_players[col].dtype}")

player_id: int64
player_url: object
fifa_version: float64
fifa_update: float64
update_as_of: object
short_name: object
long_name: object
player_positions: object
overall: int64
potential: int64
value_eur: float64
wage_eur: float64
age: int64
dob: object
height_cm: int64
weight_kg: int64
club_team_id: float64
club_name: object
league_id: float64
league_name: object
league_level: float64
club_position: object
club_jersey_number: float64
club_loaned_from: object
club_joined_date: object
club_contract_valid_until_year: float64
nationality_id: int64
nationality_name: object
nation_team_id: float64
nation_position: object
nation_jersey_number: float64
preferred_foot: object
weak_foot: int64
skill_moves: int64
international_reputation: int64
work_rate: object
body_type: object
real_face: object
release_clause_eur: float64
player_tags: object
player_traits: object
pace: float64
shooting: float64
passing: float64
dribbling: float64
defending: float64
physic: float64
attacking_crossing: int64
at

In [59]:
df_players.shape

(17954, 109)

In [60]:
all_pos = df_players.player_positions.unique()
positions = set()
for pos in all_pos:
    for p in pos.split(','):
        positions.add(p.strip())

positions = list(positions)
print(positions)

['CAM', 'CB', 'RM', 'CF', 'RB', 'GK', 'CDM', 'RWB', 'LB', 'CM', 'ST', 'RW', 'LWB', 'LW', 'LM']


In [61]:
df_players['position'] = df_players['player_positions'].apply(lambda x: x.split(',')[0].strip())

In [62]:
df_clean = df_players.copy()

# quedarse solo con las columnas relevantes

relevant_cols = ['player_id', 'short_name', 'overall', 'club_team_id','club_name', 'position', 'league_id', 'league_name', 'nationality_id','nationality_name']
df_clean = df_clean[relevant_cols]
df_clean.head()
                 

,player_id,short_name,overall,club_team_id,club_name,position,league_id,league_name,nationality_id,nationality_name
111583,20801,Cristiano Ronaldo,94,243.0,Real Madrid,LW,53.0,La Liga,38,Portugal
111584,158023,L. Messi,93,241.0,FC Barcelona,RW,53.0,La Liga,52,Argentina
111585,190871,Neymar Jr,92,73.0,Paris Saint Germain,LW,16.0,Ligue 1,54,Brazil
111586,167495,M. Neuer,92,21.0,FC Bayern München,GK,19.0,Bundesliga,21,Germany
111587,176580,L. Suárez,92,241.0,FC Barcelona,ST,53.0,La Liga,60,Uruguay


In [63]:
def quimica(player_1, player_2):
    chemistry = 0
    
    if player_1['league_id'] == player_2['league_id']:
        chemistry += 1
        if player_1['club_team_id'] == player_2['club_team_id']:
            chemistry += 1
    if player_1['nationality_id'] == player_2['nationality_id']:
        chemistry += 1
    
    return chemistry

Posibles links en una 4-3-3

In [64]:
links = {
    'GK': ['CB'],
    'RB': ['CB', 'CM'],
    'LB': ['CB', 'CM'],
    'CB': ['CB', 'CM'],
    'CM': ['CM', 'RW', 'LW', 'ST'],
    'RW': ['ST'],
    'LW': ['ST'],
}

In [ ]:
def pair_chemistry(df_i, df_j):
    # producto cartesiano
    df_i = df_i.rename(columns=lambda c: c + '_i')
    df_j = df_j.rename(columns=lambda c: c + '_j')

    df_i['key'] = 1
    df_j['key'] = 1
    pairs = df_i.merge(df_j, on='key').drop(columns='key')

    # reglas de química
    same_club   = pairs['club_team_id_i']   == pairs['club_team_id_j']
    same_league = pairs['league_id_i'] == pairs['league_id_j']
    same_nation = pairs['nationality_id_i'] == pairs['nationality_id_j']

    chem = 0
    chem = chem + same_league
    chem = chem + same_club
    chem = chem + same_nation

    pairs['chem'] = chem
    return pairs[['player_id_i', 'player_id_j', 'chem']]

results = []

for pos_i, neighs in links.items():
    df_i = df_players[df_players['position'] == pos_i]
    for pos_j in neighs:
        df_j = df_players[df_players['position'] == pos_j]
        if len(df_i) == 0 or len(df_j) == 0:
            continue
        res = pair_chemistry(df_i, df_j)
        results.append(res)

chemistry_edges = pd.concat(results, ignore_index=True)

(47396306, 3)